In [75]:
#!pip install lxml
#!pip install openai

     ---------------------------------------- 73.6/73.6 kB 1.0 MB/s eta 0:00:00


In [2]:
import json
from pprint import pprint
import os
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from lxml import etree
import json
import openai
import os
# Constants
API_SOURCE = "mediastack"
current_date, time = str(datetime.today()).split(" ")
current_date

path = os.path.join("News storage", API_SOURCE, current_date)

file_name = current_date + "_" + API_SOURCE + "_" + "extracted_news.json"
file_path = os.path.join(path, file_name)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
# OpenAI profile setup
openai.organization = "org-ha7l6qsycbULfVHdz5MwgKgr"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [132]:
regex_title = re.compile("title|headline|titular|titulo")
regex_body = re.compile("articleBody")
regex_date_published = re.compile("date.*[pP]ub.*")
regex_date_modified = re.compile("date.*[mM]od.*")
regex_tags = re.compile("tag[s]?|topic[s]?|tema[s]?|etiqueta[s]?|keyword[s]?")

def find_keys(elements_json):
    news_data = {}
    title_found = False
    body_found = False
    date_pub_found = False
    date_mod_found = False
    tag_found = False
    news_data["title"] = None
    news_data["body"] = None
    news_data["creation_date"] = None
    news_data["modification_date"] = None
    news_data["tags"] = None
    
    for element_json in elements_json:
        json_file = json.loads(element_json.get_text(), 
                               strict=False)
        if isinstance(json_file, list):
            json_file = json_file[0]
        for key in json_file:
            if not title_found and regex_title.search(key):
                news_data["title"] = json_file[key]
                title_found = True
            elif not body_found and regex_body.search(key):
                news_data["body"] = remove_tags(json_file[key])
                body_found = True
            elif not date_pub_found and regex_date_published.search(key):
                news_data["creation_date"] = json_file[key]
                date_pub_found = True
            elif not date_mod_found and regex_date_modified.search(key):
                news_data["modification_date"] = json_file[key]
                date_mod_found = True
            elif not tag_found and regex_tags.search(key):
                news_data["tags"] = json_file[key]
                data["tags"] = ";".join(data["tags"])
                tag_found = True
            if all([title_found, body_found, date_pub_found, date_mod_found, tag_found]):
                break
    return news_data
def remove_tags(text):
    return re.sub("<.*?>", "", text)

## Extracion of full article from given url

### el mundo

In [5]:
url = "https://elmundo.com"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}

In [8]:
# With headers
response = requests.get(url, headers=headers)
print(response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")

503


### El Español

In [140]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
url = "https://www.elespanol.com"
main_root = "https://www.elespanol.com"

# With headers
response = requests.get(url, headers=headers)
print("Response code:", response.status_code)  # HTTP status code
parsed_hmtl = BeautifulSoup(response.content, "html.parser")

#sections = parsed_hmtl.find_all("li", {"itemprop":"name"})
#urls_sections = [x.find("a").attrs["href"] for x in sections]
#print(urls_sections)

a_tags = parsed_hmtl.find_all("a", href=re.compile("/[0-9]+/"))

valid_news_urls = []
section_urls = []
for a in a_tags:
    try:
        href = a["href"]
        print(href)
        if not href.startswith(main_root):
            href = main_root + href
        if href.endswith("//"):
            section_urls.append(href)
        valid_news_urls.append(href)
    except:
        continue
print(len(valid_news_urls))

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

Response code: 200
/espana/politica/20230628/elecciones-generales-en-directo-ultima-hora-psoe-pp-vox-sumar-podemos-feijoo-pedro-sanchez/775022491_10.html
/espana/politica/20230628/pedro-sanchez-tacha-conservadores-programas-motos-recita-medios-apoyan/774672972_0.html
/espana/politica/20230628/feijoo-dice-informara-sueldo-deje-senado-no-plazo-fija-psoe/774922699_0.html
/espana/politica/20230628/abascal-orgullo-no-celebro-heterosexual-homosexuales/774922556_0.html
/el-cultural/cine/20230627/carmen-sevilla-folclorica-surfeo-destape-brillo-bardem-eloy-iglesia-gonzalo-suarez/774422762_0.html
/ciencia/20230627/restos-titanic-desapareciendo-bacteria-devorado-completo/774422914_0.html
/mujer/protagonistas/20230628/conoce-treinta-mujeres-espanolas-colectivo-lgtbiq-influyentes/766453359_3.html
https://www.elespanol.com/espana/politica/20230628/elecciones-generales-en-directo-ultima-hora-psoe-pp-vox-sumar-podemos-feijoo-pedro-sanchez/775022491_10.html
https://www.elespanol.com/edicion/20230628/la

ConnectionError: HTTPSConnectionPool(host='www.elespanol.comhttps', port=443): Max retries exceeded with url: //vandal.elespanol.com/avances/switch/everybody-12switch/142456/1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001A01DB844F0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

### 20minutos

In [143]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
url = "https://www.20minutos.es/"
main_root = "https://www.20minutos.es/"

date = "20230524"
# With headers
response = requests.get(url, headers=headers)
print("Response code:", response.status_code)  # HTTP status code
parsed_hmtl = BeautifulSoup(response.content, "html.parser")

#sections = parsed_hmtl.find_all("li", {"itemprop":"name"})
#urls_sections = [x.find("a").attrs["href"] for x in sections]
#print(urls_sections)

a_tags = parsed_hmtl.find_all("a", href=re.compile("/[0-9]+/"))

valid_news_urls = []
for a in a_tags:
    try:
        href = a["href"]
        print(href)
        if not href.startswith(main_root):
            href = main_root + href
        valid_news_urls.append(href)
    except:
        continue
print(len(valid_news_urls))

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"]for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)
len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

Response code: 200
https://www.20minutos.es/noticia/5142266/0/presidenta-socialista-camara-extremadura-no-espera-guardiola-convoca-investidura-miercoles-con-vara-candidato/
https://www.20minutos.es/noticia/5142146/0/directo-guerra-ucrania-rusia-bombardeos-kiev-putin-zelenski-ofensiva-militar-alerta-nuclear-ultima-hora/
https://www.20minutos.es/noticia/5142266/0/presidenta-socialista-camara-extremadura-no-espera-guardiola-convoca-investidura-miercoles-con-vara-candidato/
https://www.20minutos.es/noticia/5142185/0/abascal-insta-pp-acelerar-los-pactos-autonomicos-hemos-perdido-un-mes-sin-atender-mandato-popular/
https://www.20minutos.es/noticia/5142301/0/ocde-alerta-que-espana-vive-sequia-mas-grave-ue/
https://www.20minutos.es/noticia/5142301/0/ocde-alerta-que-espana-vive-sequia-mas-grave-ue/
https://www.20minutos.es/noticia/5142368/0/prighozin-vive-encarcelado-minsk-esta-aislado-un-hotel-sin-ventanas-para-que-no-maten/
https://www.20minutos.es/noticia/5142368/0/prighozin-vive-encarcelado

(94,
 dict_keys(['title', 'body', 'creation_date', 'modification_date', 'tags', 'url']),
 True,
 True)

### larazon

In [144]:
url = "https://www.larazon.es"
main_root = "https://www.larazon.es"

date = "20230524"
# With headers
response = requests.get(url, headers=headers)
print("Response code:", response.status_code)  # HTTP status code
parsed_hmtl = BeautifulSoup(response.content, "html.parser")

sections = parsed_hmtl.find("ul", attrs={"class": "custom_carousel_container"})
urls_sections = [x.attrs["href"] for x in sections.find_all("a")]

a_tags = parsed_hmtl.find_all("a", href=re.compile(".html$"))

valid_news_urls = []
not_valid_news_urls = []
for a in a_tags:
    try:
        href = a["href"]
        if not href.startswith(main_root):
            continue
        valid_news_urls.append(href)
        print(href)
    except:
        not_valid_news_urls.append(href)
len(valid_news_urls), len(not_valid_news_urls)

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

Response code: 200
https://www.larazon.es/cultura/adios-carmen-sevilla-eterna-novia-espana_20230627649a0334e9f5530001dc86de.html
https://www.larazon.es/sociedad/hay-fecha-retirada-mascarilla-farmacias-centros-sanitarios_20230628649bebca44049c0001eb3ac8.html
https://www.larazon.es/espana/que-diferencias-hay-buque-accion-maritima-bam-como-que-construir-armada-corbeta_20230628649c2a7fe9f5530001267e71.html
https://www.larazon.es/economia/espana-marruecos-relanzan-proyecto-tunel-estrecho-gibraltar_20230628649bea1e44049c0001eb32c8.html
https://www.larazon.es/internacional/graves-disturbios-francia-muerte-adolescente-manos-policia_20230628649bd107e9f553000124e2f7.html
https://www.larazon.es/espana/que-diferencias-hay-buque-accion-maritima-bam-como-que-construir-armada-corbeta_20230628649c2a7fe9f5530001267e71.html
https://www.larazon.es/gente/famosos/antonio-david-arropado-rocio-david-flores-juicio-rocio-carrasco_20230628649c086e44049c0001ebbece.html
https://www.larazon.es/cataluna/independent

(184,
 dict_keys(['title', 'body', 'creation_date', 'modification_date', 'tags', 'url']),
 True,
 True)

### La vanguardia

In [145]:
def find_attr_article(tag):
    return tag.has_attr('class') and "article" in " ".join(tag.attrs["class"])
def find_attr_article(tag):
    return tag.name == "article" and tag.attrs["href"]
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
url = "https://www.lavanguardia.com"
main_root = "https://www.lavanguardia.com"
date = current_date.replace("-", "")
# With headers
response = requests.get(url, headers=headers)
print("Response code:", response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")
#articles = parsed_hmtl.find_all(find_attr_article)
#raw_urls = [article.find("a").attrs["href"] for article in articles]
articles = parsed_hmtl.find_all("a", href=re.compile("/[0-9]{8}/[0-9]{1,}/"))
raw_urls = [article.attrs["href"] for article in articles]
valid_news_urls = []
rejected_urls = []
for raw_url in raw_urls:
    if not re.search(r"/[0-9]{8}/[0-9]{1,}/", raw_url):
        rejected_urls.append(raw_url)
        continue
    if raw_url.startswith("/"):
        raw_url = main_root + raw_url
    elif main_root not in raw_url:
        rejected_urls.append(raw_url)
        continue
    valid_news_urls.append(raw_url)
    print(raw_url)
valid_news_urls = list(set(valid_news_urls))

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

Response code: 200
https://www.lavanguardia.com/gente/20230628/9072453/muere-carmen-sevilla-reacciones-en-directo-hoy.html
https://www.lavanguardia.com/gente/20230628/9072627/sorpresa-atestado-policial-accidente-sergio-rico-haber-testigos-directos-mmn.html
https://www.lavanguardia.com/politica/20230628/9072866/pactos-pp-vox-movilizan-colectivo-lgtbi-dia-orgullo.html
https://www.lavanguardia.com/politica/20230628/9072849/orgullo2023-macarena-olona-tuit-dejado-indiferente-nadie.html?_gl=1*1fgs7fg*_gcl_aw*R0NMLjE2ODI4NDYxMTcuQ2p3S0NBandvN2lpQmhBRUVpd0FzSXhRRWRpWUgzZjJjRVgyYTVVMlhWWFA0bG1EeDZzWmpZZnk2VF8yTTNhRC1uR01mRURxalNSUE5Cb0NseDhRQXZEX0J3RQ..*_gcl_au*Njk5NTcyNTI5LjE2ODc2NzUzNzQ.
https://www.lavanguardia.com/television/20230628/9072661/ana-rosa-quintana-revela-dia-hora-entrevista-directo-pedro-sanchez-mmn.html
https://www.lavanguardia.com/gente/20230628/9073098/empresario-anonimo-brasil-designa-neymar-unico-heredero-fortuna-salud-mmn.html
https://www.lavanguardia.com/gente/20230628/90

(116,
 dict_keys(['title', 'body', 'creation_date', 'modification_date', 'tags', 'url']),
 True,
 True)

## Okdiario

In [147]:
def find_attr_article(tag):
    return tag.name == "article" and tag.attrs["href"]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}

url = "https://okdiario.com"
main_root = "https://okdiario.com"
date = current_date.replace("-", "")
# With headers
response = requests.get(url, headers=headers)
print("Response code:", response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")
#articles = parsed_hmtl.find_all(find_attr_article)
articles = parsed_hmtl.find_all("a", href=re.compile("-[0-9]+$"))
#raw_urls = [article.find("a").attrs["href"] for article in articles]
raw_urls = [article.attrs["href"] for article in articles]
valid_news_urls = []
rejected_urls = []
for raw_url in raw_urls:
    if not re.search(r"-[0-9]+$", raw_url):
        rejected_urls.append(raw_url)
        continue
    if raw_url.startswith("/"):
        raw_url = main_root + raw_url
    elif main_root not in raw_url:
        rejected_urls.append(raw_url)
        continue
    valid_news_urls.append(raw_url)
    print(raw_url)
valid_news_urls = list(set(valid_news_urls))
len(valid_news_urls), len(rejected_urls)

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

Response code: 200
https://okdiario.com/espana/sanchez-sale-indemne-hormiguero-habitual-despliegue-excusas-victimismo-11165348
https://okdiario.com/sociedad/sofia-suescun-sube-error-video-jugador-kings-league-semidesnudo-11165963
https://okdiario.com/deportes/podemita-fernandez-marca-ridiculo-historico-acusar-florentino-apoyar-ultra-sur-11162607
https://okdiario.com/sociedad/mario-picazo-nos-revienta-fin-semana-peor-noticia-vuelven-lluvias-espana-11166256
https://okdiario.com/elecciones/buscador-interactivo-consulta-aqui-si-te-tocado-estar-mesa-electoral-11160941
https://okdiario.com/espana/sanchez-cobrado-48-664-dietas-del-congreso-vivienda-pese-residir-moncloa-11132471
https://okdiario.com/happyfm/entrevista-alberto-nunez-feijoo-hormiguero-que-hora-como-verla-directo-11160807
https://okdiario.com/comunidad-valenciana/mazon-exige-ximo-puig-que-marche-vez-deje-gobernar-presidente-salga-del-sillon-11160759
https://okdiario.com/curiosidades/apunta-este-destino-irte-vacaciones-pais-menos-

(215,
 dict_keys(['title', 'body', 'creation_date', 'modification_date', 'tags', 'url']),
 True,
 True)

## ABC

In [151]:
url = "https://abc.es"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
main_root = "https://www.abc.es"
# With headers
response = requests.get(url, headers=headers)
print(response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")

a_tags = parsed_hmtl.find_all("a", href=re.compile("-[a-z]{1,2}.html$"))
valid_news_urls = []
not_valid_news_urls = []
for a in a_tags:
    try:
        href = a["href"]
        if main_root not in href:
            href = main_root + href
        valid_news_urls.append(href)
        print(href)
    except:
        continue
print(len(valid_news_urls))

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

200
https://www.abc.es/sociedad/aemet-avisa-cambio-radical-tiempo-espana-llega-frente-frio-tras-ola-calor-20230628161519-nt.html
https://www.abc.es/gente/millonaria-herencia-carmen-sevilla-20230628120341-nt.html
https://www.abc.es/play/television/noticias/pedro-sanchez-defiende-tajante-ataques-hormiguero-adelanto-20230627230358-nt.html
https://www.abc.es/familia/padres-hijos/semanas-permiso-cuidar-tus-hijos-remunerar-nuevas-20230628130553-nt.html
https://www.abc.es/economia/excepciones-fin-llamadas-spam-casos-pueden-seguir-20230628124348-nt.html
https://www.abc.es/economia/carta-imserso-recibiran-pensionistas-proximos-dias-20230628135946-nt.html
https://www.abc.es/espana/elecciones-directo-ultima-hora-sanchez-hormiguero-feijoo-diaz-actos-precampana-hoy-20230627161355-di.html
https://www.abc.es/espana/madrid/falsa-casera-alquilaba-piso-inquilinos-vez-20230628040322-nt.html
https://www.abc.es/play/television/noticias/susanna-griso-defiende-pablo-motos-actitud-pedro-20230628095907-nt.html

(85,
 dict_keys(['title', 'body', 'creation_date', 'modification_date', 'tags', 'url']),
 True,
 True)

### Vandal

In [149]:
url = "https://vandal.elespanol.com"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
main_root = "https://vandal.elespanol.com"
# With headers
response = requests.get(url, headers=headers)
print(response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")

a_tags = parsed_hmtl.find_all("a", href=re.compile("/[0-9]+/"))
valid_news_urls = []
not_valid_news_urls = []
for a in a_tags:
    try:
        href = a["href"]
        if main_root not in href:
            href = main_root + href
        valid_news_urls.append(href)
        print(href)
    except:
        continue
print(len(valid_news_urls))

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if not data["tags"]:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    if not data["tags"]:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"name": "keywords"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

200
https://vandal.elespanol.com/noticia/1350763526/filtrados-los-juegos-de-ps-plus-de-julio-alan-wake-remastered-cod-bo-cold-war-y-endling/
https://vandal.elespanol.com/noticia/1350763495/el-jefe-de-xbox-games-studios-estaba-listo-para-acabar-con-sony-a-golpe-de-talonario/
https://vandal.elespanol.com/juegos/0/videojuegos
https://vandal.elespanol.com/lanzamientos/0/videojuegos
https://vandal.elespanol.com/avances/pc/ride-5/133942/1
https://vandal.elespanol.com/avances/pc/hot-wheels-unleashed-2-turbocharged/142305/1
https://vandal.elespanol.com/noticia/1350763522/final-fantasy-16-supera-los-3-millones-de-copias-vendidas-en-los-primeros-dias/
https://vandal.elespanol.com/noticia/1350763512/red-dead-redemption-remastered-el-clasico-de-rockstar-aparece-en-una-clasificacion-por-edades/
https://vandal.elespanol.com/noticia/1350763502/nintendo-quiere-evitar-la-especulacion-con-la-sucesora-de-switch-fabricando-mas-consolas/
https://vandal.elespanol.com/noticia/1350763518/scorn-el-juego-de-ter

(65,
 dict_keys(['title', 'body', 'creation_date', 'modification_date', 'tags', 'url']),
 True,
 True)